In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Output file name
outputFile = 'ICE Current vs Projected Rebalance.xlsx'

# Total market Cap for PFF, needs to be manually updated
# Can be automated later
current_pff_market_cap = 13708086021

# Reading the ICE Data for Current and Projected Universe
ice_currentUniverse_df = pd.read_excel('Data/PHGY-Current.xlsx', skiprows=[0])
ice_projectedUniverse_df = pd.read_excel('Data/PHGY-Projected.xlsx', skiprows=[0])

# DataFrame for our final Output
pff_df = pd.DataFrame()

In [3]:
# When reading data, below strings are read, they must be marked as np.nan
msg1 = 'Any unauthorized use or disclosure is prohibited. Nothing herein should in any way be deemed to alter the legal rights and obligations contained in agreements between any ICE Data Services entity ("ICE") and their clients relating to any of the Indices or products or services described herein. The information provided by ICE and contained herein is subject to change without notice and does not constitute any form of representation, or undertaking.  ICE and its affiliates make no warranties whatsoever, either express or implied, as to merchantability, fitness for a particular purpose, or any other matter in connection with the information provided. Without limiting the foregoing, ICE and its affiliates makes no representation or warranty that any information provided hereunder are complete or free from errors, omissions, or defects. All information provided by ICE is owned by or licensed to ICE. ICE retains exclusive ownership of the ICE Indices, including the ICE BofAML Indexes, and the analytics used to create this analysis ICE may in its absolute discretion and without prior notice revise or terminate the ICE information and analytics at any time. The information in this analysis is for internal use only and redistribution of this information to third parties is expressly prohibited.'
msg2 = 'Neither the analysis nor the information contained therein constitutes investment advice or an offer  or an invitation to make an offer  to buy or sell any securities or any options  futures or other derivatives related to such securities. The information and calculations contained in this analysis have been obtained from a variety of sources  including those other than ICE and ICE does not guarantee their accuracy.  Prior to relying on any ICE information and/or the execution of a security trade based upon such ICE information, you are advised to consult with your broker or other financial representative to verify pricing information. There is no assurance that hypothetical results will be equal to actual performance under any market conditions. THE ICE INFORMATION IS PROVIDED TO THE USERS "AS IS." NEITHER ICE, NOR ITS AFFILIATES, NOR ANY THIRD PARTY DATA PROVIDER WILL BE LIABLE TO ANY USER OR ANYONE ELSE FOR ANY INTERRUPTION, INACCURACY, ERROR OR OMISSION, REGARDLESS OF CAUSE, IN THE ICE INFORMATION OR FOR ANY DAMAGES RESULTING THEREFROM. In no event shall ICE or any of its affiliates, employees  officers  directors or agents of any such persons have any liability to any person or entity relating to or arising out of this information, analysis  or the indices  contained herein.'

In [4]:
# Current Universe Data
ice_currentUniverse_df = ice_currentUniverse_df.replace('NaN', np.nan)
ice_currentUniverse_df = ice_currentUniverse_df.replace(msg1, np.nan)
ice_currentUniverse_df= ice_currentUniverse_df.replace(msg2, np.nan)

#drop row if ISIN number is Nan
ice_currentUniverse_df.dropna(subset=['ISIN number'], inplace=True)

# Projected Universe Data
ice_projectedUniverse_df = ice_projectedUniverse_df.replace('NaN', np.nan)
ice_projectedUniverse_df = ice_projectedUniverse_df.replace(msg1, np.nan)
ice_projectedUniverse_df= ice_projectedUniverse_df.replace(msg2, np.nan)

#drop row if ISIN number is Nan
ice_projectedUniverse_df.dropna(subset=['ISIN number'], inplace=True)

In [5]:
# Storing the unique ISIN ID's from both the current and projected universe data
unique_ISIN = pd.concat([ice_currentUniverse_df['ISIN number'], ice_projectedUniverse_df['ISIN number']]).drop_duplicates().reset_index(drop=True)
pff_df['ISIN number'] = unique_ISIN

In [6]:
isin_to_ticker_df = pd.read_excel('Static Data/ISINtoTicker.xlsx')
pff_df = pff_df.merge(isin_to_ticker_df, left_on='ISIN number', right_on='ISIN', how='left').drop(['ISIN'], axis=1)

In [7]:
pff_df['Activ Ticker'] = pff_df['Ticker'].str.replace(".PR","p")

/var/folders/3f/bwblj7n13596fl7z3cp9q_c40000gn/T/ipykernel_13426/2335501598.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  pff_df['Activ Ticker'] = pff_df['Ticker'].str.replace(".PR","p")


In [8]:
# getting the ActivDiv for reading the prices
activ_df = pd.read_excel('Static Data/ACtivDivData.xlsx', 'PasteByValues')
activ_df = activ_df[['Activ Ticker', 'Last Price']]

In [9]:
def getLastPrice(x):
    activTicker = x['Activ Ticker']
    ticker = activ_df[activ_df['Activ Ticker']==activTicker]
    if(len(ticker)>0):
        return ticker.iloc[0]['Last Price']
    return np.nan

pff_df['Last Price'] = pff_df.apply(getLastPrice, axis=1)

In [10]:
def getCurrentMarketCap(x):
    isid = x['ISIN number']
    ticker = ice_currentUniverse_df[ice_currentUniverse_df['ISIN number']==isid]
    if(len(ticker)>0):
        return ticker.iloc[0]['% Mkt Value']
    return 0

def getProjectedMarketCap(x):
    isid = x['ISIN number']
    ticker = ice_projectedUniverse_df[ice_projectedUniverse_df['ISIN number']==isid]
    if(len(ticker)>0):
        return ticker.iloc[0]['% Mkt Value']
    return 0

pff_df['Current % Mkt Cap'] = pff_df.apply(getCurrentMarketCap, axis=1)
pff_df['Projected % Mkt Cap'] = pff_df.apply(getProjectedMarketCap, axis=1)

In [11]:
def getCurrentPFFShares(x):
    if(not isinstance(x['Last Price'], str)):
        return np.rint((current_pff_market_cap*x['Current % Mkt Cap'])/(x['Last Price']*100))
    return np.nan

def getProjectedPFFShares(x):
    if(not isinstance(x['Last Price'], str)):
        return np.rint((current_pff_market_cap*x['Projected % Mkt Cap'])/(x['Last Price']*100))
    return np.nan

pff_df['Current PFF Shares'] = pff_df.apply(getCurrentPFFShares, axis=1)
pff_df['Projected PFF Shares'] = pff_df.apply(getProjectedPFFShares, axis=1)
pff_df['Difference'] = pff_df.apply(lambda x: x['Projected PFF Shares']-x['Current PFF Shares'], axis=1)

In [12]:
total_buys_ice = pff_df['Difference'].where(pff_df['Difference']>0).sum()
total_sells_ice = pff_df['Difference'].where(pff_df['Difference']<0).sum()
total_pff_transactions_df = pd.DataFrame({"Number of Shares":[total_buys_ice, total_sells_ice]}, index=['Total PFF Buys', 'Total PFF Sells'])

In [13]:
with pd.ExcelWriter(outputFile) as writer:
    pff_df.to_excel(writer, sheet_name='ICE Current vs ICE Projected')
    total_pff_transactions_df.to_excel(writer, sheet_name='Total PFF Transactions')

In [14]:
pff_df[pff_df['Last Price'].apply(lambda x: (type(x)!=float and type(x)!=int) or math.isnan(x))].drop(110)

,ISIN number,Ticker,Activ Ticker,Last Price,Current % Mkt Cap,Projected % Mkt Cap,Current PFF Shares,Projected PFF Shares,Difference
101,US1127142099,DTLA,DTLA,UD,0.01668,0.01708,NaN,NaN,NaN
144,US19247G2066,IIVIP,IIVIP,UD,0.24260,0.00000,NaN,NaN,NaN
179,US26884U2087,EPR.PRC,ppC,source not found,0.05843,0.05981,NaN,NaN,NaN
180,US26884U5056,EPR.PRG,ppG,source not found,0.06466,0.06619,NaN,NaN,NaN
181,US26884U3077,EPR.PRE,ppE,NaN,0.05315,0.05440,NaN,NaN,NaN
399,US8385182071,SJIJ,SJIJ,target not permissioned,0.07219,0.00000,NaN,NaN,NaN
400,US8385183061,SJIV,SJIV,target not permissioned,0.24614,0.00000,NaN,NaN,NaN
